In [181]:
import numpy as np

import pandas as pd

import sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,accuracy_score
from sklearn.model_selection import learning_curve

from feature_engine.datetime import DatetimeFeatures

from xgboost import XGBRegressor

import joblib

import matplotlib.pyplot as plt

In [182]:
sklearn.set_config(transform_output="default")

In [183]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [184]:
train_df = train_df[train_df['Duration'].str.contains('h')]

In [185]:
def timer(row):
    dx=row.split(" ")
    if dx[0]!=dx[-1]:
        dy=int(dx[-1].split("m")[0])
    else:
        dy=0    
    
    return int(dx[0].split("h")[0])*60+ dy

In [62]:
x=train_df["Duration"][8547].split(" ")
if x[0]!=x[-1]:
        dy=int(x[-1].split("m")[0])
else:
        dy=0
int(x[0].split("h")[0])*60+dy

1080

In [45]:
x=train_df["Duration"][8547].split(" ")
if x[0]!=x[-1]:
    print(int(x[1].split("m")[0]))
else:
    print(0)    

0


In [186]:
train_df["Duration_mins"]=train_df["Duration"].apply(timer)

In [187]:
def solve(row):
    if row=="No info":
        return 0
    else:
        return 1

In [188]:
train_df["Additional_Info_upd"]=train_df["Additional_Info"].apply(solve)

In [189]:
def stopie(row):
    if row=="non-stop":
        return 0
    
    else:
        return row.split(" ")[0]

In [190]:
train_df["Total_Stops_upd"]=train_df["Total_Stops"].apply(stopie)

In [195]:
train_df

,Airline,Source,Destination,Additional_Info_upd,Duration_mins,Total_Stops_upd,Price
0,IndiGo,Banglore,New Delhi,0,170,0,3897
1,Air India,Kolkata,Banglore,0,445,2,7662
2,Jet Airways,Delhi,Cochin,0,1140,2,13882
3,IndiGo,Kolkata,Banglore,0,325,1,6218
4,IndiGo,Banglore,New Delhi,0,285,1,13302
...,...,...,...,...,...,...,...
8542,Jet Airways,Kolkata,Banglore,1,800,1,10844
8543,SpiceJet,Kolkata,Banglore,1,145,0,3841
8544,Jet Airways,Kolkata,Banglore,1,590,1,10844
8545,Jet Airways,Delhi,Cochin,1,820,2,8728


In [192]:

test_df["Additional_Info_upd"]=test_df["Additional_Info"].apply(solve)
test_df["Duration_mins"]=test_df["Duration"].apply(timer)

In [193]:
def stopie(row):
    # Check if the value is a float (NaN values will also be floats)
    if row=="non-stop":
        return 0
    
    elif isinstance(row, float):
        return 0
    else:
        # Split the string and convert the first part to an integer
        return int(row.split(" ")[0])

# Apply the function to the 'Total_Stops' column
test_df["Total_Stops_upd"] = test_df["Total_Stops"].apply(stopie)

In [194]:
train_df=train_df[["Airline","Source","Destination","Additional_Info_upd","Duration_mins","Total_Stops_upd","Price"]]
test_df=test_df[["Airline","Source","Destination","Additional_Info_upd","Duration_mins","Total_Stops_upd","Price"]]

In [180]:
train_df


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
8542,Jet Airways,9/05/2019,Kolkata,Banglore,CCU → BOM → BLR,06:30,19:50,13h 20m,1 stop,In-flight meal not included,10844
8543,SpiceJet,24/05/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No check-in baggage included,3841
8544,Jet Airways,15/05/2019,Kolkata,Banglore,CCU → BOM → BLR,06:30,16:20,9h 50m,1 stop,In-flight meal not included,10844
8545,Jet Airways,27/03/2019,Delhi,Cochin,DEL → MAA → BOM → COK,05:10,18:50,13h 40m,2 stops,In-flight meal not included,8728


In [196]:
def split_data(data):
	x = data.drop(columns="Price")
	y = data["Price"].values
	return (x, y)

In [197]:
X_train, y_train = split_data(train_df)

In [198]:
X_train

,Airline,Source,Destination,Additional_Info_upd,Duration_mins,Total_Stops_upd
0,IndiGo,Banglore,New Delhi,0,170,0
1,Air India,Kolkata,Banglore,0,445,2
2,Jet Airways,Delhi,Cochin,0,1140,2
3,IndiGo,Kolkata,Banglore,0,325,1
4,IndiGo,Banglore,New Delhi,0,285,1
...,...,...,...,...,...,...
8542,Jet Airways,Kolkata,Banglore,1,800,1
8543,SpiceJet,Kolkata,Banglore,1,145,0
8544,Jet Airways,Kolkata,Banglore,1,590,1
8545,Jet Airways,Delhi,Cochin,1,820,2


In [199]:
X_test, y_test = split_data(test_df)

In [200]:
num_cols = ["Duration_mins", "Total_Stops_upd","Additional_Info_upd"]

cat_cols = [col for col in X_train.columns if  (col not in num_cols)]

In [201]:
cat_cols

['Airline', 'Source', 'Destination']

In [202]:
num_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="median")),
	("scaler", StandardScaler())
])

cat_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

In [203]:
preprocessor = ColumnTransformer(transformers=[
	("num", num_transformer, num_cols),
	("cat", cat_transformer, cat_cols),
	
])

In [204]:
preprocessor.fit_transform(X_train)
preprocessor.transform(X_test)

array([[-0.34542698,  0.26717313, -0.52472092, ...,  0.        ,
         0.        ,  0.        ],
       [-0.97891169, -1.21835813, -0.52472092, ...,  0.        ,
         0.        ,  0.        ],
       [-0.36522337,  0.26717313, -0.52472092, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.9096243 , -1.21835813, -0.52472092, ...,  0.        ,
         0.        ,  0.        ],
       [-0.94921709, -1.21835813, -0.52472092, ...,  0.        ,
         0.        ,  1.        ],
       [-0.27613959,  1.75270438, -0.52472092, ...,  0.        ,
         0.        ,  0.        ]])

In [205]:
algorithms = {
	"Linear Regression": LinearRegression(),
	"Random Forest": RandomForestRegressor(n_estimators=10),
	"XG Boost": XGBRegressor(n_estimators=10)
}

In [206]:
model = Pipeline(steps=[
	("pre", preprocessor),
	("rf", XGBRegressor(n_estimators=10))
])

In [207]:
model.fit(X_train,y_train)

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Duration_mins',
                                                   'Total_Stops_upd',
                                                   'Additional_Info_upd']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=10, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [208]:
preds=model.predict(X_test)

In [220]:
preds

array([10616.503 ,  4324.401 , 10616.503 , ...,  7261.8657,  7912.0767,
       12512.632 ], dtype=float32)

In [209]:
r2_score(y_test,preds)

0.7036395157722444

In [210]:
joblib.dump(model, "model.joblib")

['model.joblib']

In [211]:
saved_model = joblib.load("model.joblib")
saved_model

Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Duration_mins',
                                                   'Total_Stops_upd',
                                                   'Additional_Info_upd']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=10, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [212]:
y_pred = saved_model.predict(X_test)

r2_score(y_test, y_pred)

0.7036395157722444

In [213]:
train_df

,Airline,Source,Destination,Additional_Info_upd,Duration_mins,Total_Stops_upd,Price
0,IndiGo,Banglore,New Delhi,0,170,0,3897
1,Air India,Kolkata,Banglore,0,445,2,7662
2,Jet Airways,Delhi,Cochin,0,1140,2,13882
3,IndiGo,Kolkata,Banglore,0,325,1,6218
4,IndiGo,Banglore,New Delhi,0,285,1,13302
...,...,...,...,...,...,...,...
8542,Jet Airways,Kolkata,Banglore,1,800,1,10844
8543,SpiceJet,Kolkata,Banglore,1,145,0,3841
8544,Jet Airways,Kolkata,Banglore,1,590,1,10844
8545,Jet Airways,Delhi,Cochin,1,820,2,8728


In [214]:
import pandas as pd

# Assuming train_df is already defined and contains the necessary columns
selected_columns = ["Airline", "Source", "Destination", "Additional_Info_upd", "Duration_mins", "Total_Stops_upd", "Price"]
train_df_selected = train_df[selected_columns]

# Save to CSV
train_df_selected.to_csv("selected_features_train_data.csv", index=False)


In [215]:
test_df_selected = test_df[selected_columns]

# Save to CSV
test_df_selected.to_csv("selected_features_test_data.csv", index=False)